In [1]:
# from data_gen import *
from models import *
from heb_proc import *

Using TensorFlow backend.


In [2]:

featMap, root2feat, root2word = read(file='data/heb/hebrew-task1-train')
char2int, int2char, max_root, max_word, feat_length = stat(root2feat, root2word, featMap)
print(max_word, max_root, feat_length)
roots_mat, decoder_words_mat, feats_vec, out_words_mat = get_data(root2word, root2feat, char2int, max_root, max_word, feat_length)
batch_size = 100
n_input_length = roots_mat.shape[1]
n_output_shape = out_words_mat.shape[1]
n_feature = feat_length
n_batches = len(root2feat) // batch_size
print("Root shape: ", roots_mat.shape)
print("Decoder input shape: ", decoder_words_mat.shape)
print("Feature shape: ", feats_vec.shape)
print("Output shape: ", out_words_mat.shape)


15 15 44
Root shape:  (7326, 15, 31)
Decoder input shape:  (7326, 17, 31)
Feature shape:  (7326, 44)
Output shape:  (7326, 17, 31)


In [3]:
gen = gen_data(roots_mat, decoder_words_mat, feats_vec, out_words_mat)
# (roots, decode, feat), words = next(gen)

In [4]:
# words.shape

In [5]:
# gen = gen_data(roots_mat, words_mat, feats_vec, batch_size=batch_size)

In [6]:
root_h, root_w = roots_mat.shape[1], roots_mat.shape[2]
decoder_input = decoder_words_mat.shape[2]
decoder_output = out_words_mat.shape[1]
feat_length = feats_vec.shape[1]
model, encoder, decoder = conv_model(root_h, root_w, decoder_input, decoder_output, feat_length, 512, feat_units=20)
model.compile(optimizer='adam', loss='categorical_crossentropy')


In [7]:
# model.summary()

In [8]:
# model train given the data generator, how many batches and number of epochs
history = model.fit_generator(gen, steps_per_epoch=n_batches, epochs = 45)

Epoch 1/45
73/73 [==============================] - 13s 175ms/step - loss: 1.4807
Epoch 2/45
73/73 [==============================] - 8s 109ms/step - loss: 1.0656
Epoch 3/45
73/73 [==============================] - 8s 108ms/step - loss: 0.9111
Epoch 4/45
73/73 [==============================] - 8s 108ms/step - loss: 0.7548
Epoch 5/45
73/73 [==============================] - 8s 108ms/step - loss: 0.5913
Epoch 6/45
73/73 [==============================] - 8s 108ms/step - loss: 0.4381
Epoch 7/45
73/73 [==============================] - 8s 108ms/step - loss: 0.3222
Epoch 8/45
73/73 [==============================] - 8s 108ms/step - loss: 0.2477
Epoch 9/45
73/73 [==============================] - 8s 108ms/step - loss: 0.1919
Epoch 10/45
73/73 [==============================] - 8s 108ms/step - loss: 0.1494
Epoch 11/45
73/73 [==============================] - 8s 108ms/step - loss: 0.1227
Epoch 12/45
73/73 [==============================] - 8s 108ms/step - loss: 0.1039
Epoch 13/45
73/73 [=====

In [9]:
tfeat_map, troot2feat, troot2word = read('data/heb/hebrew-task1-test', featMap)
troots_mat, tdecoder_words_mat, tfeats_vec, tout_words_mat = get_data(troot2word, troot2feat, char2int, max_root, max_word, feat_length)
test_gen = gen_data(troots_mat, tdecoder_words_mat, tfeats_vec, tout_words_mat)

In [10]:
def predict(encoder, decoder, root_word, word_feat, max_step, output_dim, char2int):
    # encode
    state = encoder.predict([root_word, word_feat])
    # start of sequence input
    start = [0.0 for _ in range(output_dim)]
    start[char2int['&']] = 1
    target_seq = np.array(start).reshape(1, 1, output_dim)
    # collect predictions
    output = list()
    for t in range(max_step):
        # predict next char
        yhat, h= decoder.predict([target_seq, state])
        # store prediction
        output.append(yhat[0,0,:])
        # update state
        state = h
        # update target sequence
        target_seq = yhat
    return np.array(output)

In [11]:
test_n_batches = 30 #len(root2feat) // batch_size

In [12]:
# shows sample examples and calculates accuracy

total, correct = 0, 0
in_word = 0
sims = []
for b in range(test_n_batches):
    # get data from test data generator
    [roots, feats, words], y = next(test_gen)
    for j in range(batch_size):
        word_features = feats[j].reshape((1, feats.shape[1])) 
        root_word_matrix = roots[j].reshape((1, roots.shape[1], roots.shape[2], 1))
        
        # predicts the target word given root word and features
        target = predict(encoder, decoder, root_word_matrix, word_features, max_word, decoder_input, char2int)
        root = ''.join(one_hot_decode(roots[j], int2char))#.replace('&', ' ')
        word = ''.join(one_hot_decode(y[j], int2char))#.replace('&', ' ')
        targetS = ''.join(one_hot_decode(target, int2char))#.replace('&', ' ')
#         sims.append(dg.word_sim(word, targetS))
        target =  target[:-1]
        # checks if the predicted and the real words are equal
        if one_hot_decode(y[j], int2char) == one_hot_decode(target, int2char):
            correct += 1
#         else:
#             print(root, word.split('&')[0], '\t\t', targetS.split('&')[0])
#         if root.strip() in targetS.strip():
#             in_word += 1
#     print(b, root, word, targetS)
    total += batch_size
    

print('Exact Accuracy: %.2f%%' % (float(correct)/float(total)*100.0))

Exact Accuracy: 0.00%
